In [11]:
import pandas as pd
from datetime import datetime
df = pd.read_csv('/content/Nat_Gas (1).csv')
df.rename(columns={'Dates': 'Date', 'Prices': 'Price'}, inplace=True)

injection_options = [
    '2023-08-31', '2023-09-30',
    '2023-10-31', '2023-11-30',
    '2024-01-31', '2024-03-31'
]

withdrawal_date = '2024-09-30'

print("Injection Strategy Evaluation:\n")
for inject_date in injection_options:
    value = price_storage_contract(
        market_df=df,
        injection_dates=[inject_date],
        withdrawal_dates=[withdrawal_date],
        injection_rate=10,
        withdrawal_rate=10,
        max_storage_volume=50,
        storage_cost_per_mmbtu_per_day=0.05,
        verbose=False
    )
    print(f" Inject on {inject_date} → Withdraw on {withdrawal_date} → Contract Value: ${value}")


value = price_storage_contract(
    market_df=df,
    injection_dates=['2023-07-31', '2023-09-30'],
    withdrawal_dates=['2023-12-31', '2024-03-31'],
    injection_rate=10,
    withdrawal_rate=10,
    max_storage_volume=50,
    storage_cost_per_mmbtu_per_day=0.05
)

print(f"  Estimated Contract Value: ${value}")


def price_storage_contract(
    market_df,
    injection_dates,
    withdrawal_dates,
    injection_rate,
    withdrawal_rate,
    max_storage_volume,
    storage_cost_per_mmbtu_per_day,
    verbose=True
):
    market_df['Date'] = pd.to_datetime(market_df['Date'])
    market_df['Price'] = market_df['Price'].astype(float)
    price_lookup = market_df.set_index(market_df['Date'].dt.strftime('%Y-%m-%d'))['Price'].to_dict()

    total_volume = 0
    total_cost = 0
    total_revenue = 0
    total_storage_fee = 0

    for inj_date, wdr_date in zip(injection_dates, withdrawal_dates):
        inj_date = pd.to_datetime(inj_date)
        wdr_date = pd.to_datetime(wdr_date)

        inj_key = inj_date.strftime('%Y-%m-%d')
        wdr_key = wdr_date.strftime('%Y-%m-%d')


        if wdr_date <= inj_date:
            raise ValueError(f" Withdrawal date {wdr_date.date()} must be after injection date {inj_date.date()}")
        if inj_key not in price_lookup or wdr_key not in price_lookup:
            raise ValueError(f"Missing price for one of the dates: {inj_key} or {wdr_key}")

        inj_price = price_lookup[inj_key]
        wdr_price = price_lookup[wdr_key]

        total_volume += injection_rate
        if total_volume > max_storage_volume:
            raise ValueError(f"Storage capacity exceeded. Max allowed: {max_storage_volume} MMBtu")

        cost = injection_rate * inj_price
        revenue = withdrawal_rate * wdr_price
        days_stored = (wdr_date - inj_date).days
        storage_fee = injection_rate * storage_cost_per_mmbtu_per_day * days_stored

        total_cost += cost
        total_revenue += revenue
        total_storage_fee += storage_fee

        if verbose:
            print(f"Injected {injection_rate} MMBtu on {inj_date.date()} at ${inj_price:.2f}")
            print(f"Withdrew {withdrawal_rate} MMBtu on {wdr_date.date()} at ${wdr_price:.2f}")
            print(f"Duration Period {days_stored} days → Storage Cost: ${storage_fee:.2f}")
            print(".")

    final_value = total_revenue - total_cost - total_storage_fee
    return round(final_value, 2)

Injection Strategy Evaluation:

 Inject on 2023-08-31 → Withdraw on 2024-09-30 → Contract Value: $-191.0
 Inject on 2023-09-30 → Withdraw on 2024-09-30 → Contract Value: $-180.0
 Inject on 2023-10-31 → Withdraw on 2024-09-30 → Contract Value: $-167.5
 Inject on 2023-11-30 → Withdraw on 2024-09-30 → Contract Value: $-156.5
 Inject on 2024-01-31 → Withdraw on 2024-09-30 → Contract Value: $-129.5
 Inject on 2024-03-31 → Withdraw on 2024-09-30 → Contract Value: $-100.5
Injected 10 MMBtu on 2023-07-31 at $11.40
Withdrew 10 MMBtu on 2023-12-31 at $12.80
Duration Period 153 days → Storage Cost: $76.50
.
Injected 10 MMBtu on 2023-09-30 at $11.50
Withdrew 10 MMBtu on 2024-03-31 at $12.70
Duration Period 183 days → Storage Cost: $91.50
.
  Estimated Contract Value: $-142.0


/tmp/ipython-input-10-767821541.py:52: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  market_df['Date'] = pd.to_datetime(market_df['Date'])
